In [ ]:
# passivate/activate method
# https://www.salabim.org/manual/Modelling.html#a-bank-example

import salabim as sim


class CarGenerator(sim.Component):
    def setup(self):
        self.mode.monitor(False)
        self.status.monitor(False) # added 17 nov, salabim team tip

    def process(self):
        while True:
            Car()
            self.hold(iat_distr.sample())


class Car(sim.Component):
    def setup(self):
        self.mode.monitor(False)
        self.status.monitor(False) # added 17 nov, salabim team tip

    def process(self):
        self.enter(waitingline)
        for ChargingStation in ChargingStations:
            if ChargingStation.ispassive():
                ChargingStation.activate()
                break  # activate at most one charging station
        self.passivate()


class ChargingStation(sim.Component):
    def setup(self):
        self.mode.monitor(False)
        self.status.monitor(False)  # added 17 nov, salabim team tip

    def process(self):
        while True:
            while len(waitingline) == 0:
                self.passivate()
            self.car = waitingline.pop()
            self.hold(srv_distr.sample())
            self.car.activate()


N_STATION = 1
iat_distr = sim.Exponential(60 / 40)
srv_distr = sim.Exponential(60 / 50)

# https://www.salabim.org/manual/Reference.html#environment
app = sim.App(
    trace=False,  # defines whether to trace or not
    random_seed="*",  # if “*”, a purely random value (based on the current time)
    time_unit="minutes",  # defines the time unit used in the simulation
    name="Charging Station",  # name of the simulation
    do_reset=True,  # defines whether to reset the simulation when the run method is called
    yieldless=True,  # defines whether the simulation is yieldless or not
)

# Instantiate and activate the client generator
CarGenerator(name="Electric Cars Generator")

# Create Queue and set monitor to stats_only
waitingline = sim.Queue(name="Waiting Cars", monitor=False)
waitingline.length_of_stay.monitor(value=False)
# waitingline.length_of_stay.reset_monitors(stats_only=True)

# Instantiate the servers, list comprehension but only 1 server
ChargingStations = [ChargingStation() for _ in range(N_STATION)]

# Execute Simulation
app.run(till=sim.inf)

# Print statistics
waitingline.length_of_stay.print_statistics()